# PITCH TRACKING VISUALIZATION SOFTWARE
by Blake Harrison

This project aims to provide a visualization of pitches thrown in Major League Baseball games. The program takes in data from MLB's [Baseball Savant](https://baseballsavant.mlb.com/statcast_search) website. This website can be used to download CSV data files that contain all the necessary metrics.

For the purposes of demonstration, this repository contains data files from three major league pitchers: Clayton Kershaw, of the Los Angeles Dodgers (kershaw_data.csv), Kenta Maeda, of the Minnesota Twins (maeda_data.csv), and Aroldis Chapman, of the New York Yankees (chapman_data.csv).

## Physics
The data gives us several key pieces of information about each pitch. We can turn that information into a visualization of the pitch in motion.

We get, from the data:
- The position of the pitch (x, y, and z) when the pitcher releases it
- The velocity of the pitch (in the x-, y- and z-directions) when the pitcher releases it
- The spin axis (in degrees)
- The spin rate at release (in rpm)

The coordinate system used by this data is different than what we would normally expect, however.
The back left corner of home plate is the origin (0,0).
The positive x-direction is to the catcher's right.
The positive y-direction is from home plate towards second base.
The positive z-direction is upwards.

The spin axis represents degrees in the 2D X-Z plane, such that 180 degrees represents pure topspin, and 0 degrees represents pure backspin. We can also assert that 90 degrees is clockwise sidespin (from the catcher's perspective) and 270 degrees is counterclockwise sidespin (also from the catcher's perspective).

To break our spin into x- and z- components, we can use trigonometry. With an axis $\theta$, our x-component will be $sin(\theta)$, and our z-component will be $cos(\theta)$.

## Magnus Effect

The magnus effect describes the force on the ball by the air as a result of the ball's spin. That is calculated as:

$$\vec{F}_{magnus} = \alpha \vec{\omega} \times \vec{v}$$

The spin parameter is defined:

$$S = \frac{r\omega}{v}$$


The magnus force is written:

$$\vec{F}_{magnus} = \frac{1}{2}C_L \frac{\rho A r}{S} \vec{\omega} \times \vec{v}$$

The lift coefficient $C_L$ depends on the spin factor and is determined by a curve fit to data.

$$C_L = 0.62S^{0.7}$$


## Packages
This program uses several packages.

ode is used to run ordinary differential equations.
Numpy is used for several arrays.
Vpython provides the visualization.
CSV reads csv files, which is where the input comes from.

In [1]:
import ode
import numpy as np
from vpython import *
import csv

<IPython.core.display.Javascript object>

## Data Set 1: Clayton Kershaw, 04/01/21
This first data set uses pitch data from Los Angeles Dodger Pitcher Clayton Kershaw. This game was against the Colorado Rockies.

This model will show all eleven pitches he threw in the first inning (against four batters). 

The first three against Ramiel Tapia were all four-seam fastballs.
The next pitch, against Josh Fuentes, was also a four-seam fastball.
The next batter was Trevor Story, and Kershaw threw, in order, a four-seam fastball, a slider, a changeup, and a four-seam fastball.
The last batter he faced was Charlie Blackmon, and he threw a slider, a changeup, and a four-seam fastball.

In [2]:
def readData(filename = "kershaw_data.csv"):
    #default dataset is savant.csv, user can pass in whatever they want
    
    #creates the necessary lists
    date = [] 
    release_speed = [] #in mph
    release_pos = [] #vectors of x0, y0, v0
    player_name = []
    event = []
    result = [] #result of the pitch
    balls = [] #current # of balls
    strikes = [] #current # of strikes
    pitch_name = [] #pitch type
    release_vel = [] #vectors of vx0, vy0, and vz0
    spin_axis = [] #spin axis in 2D x-z plane, such that 180 is pure backspin and 0 is pure topspin
    release_spin = [] #spin rate of pitch (rpm)
    
    with open(filename) as file:
        rawdata = csv.reader(file, delimiter = ',')
        next(file)
        for line in rawdata:
            date.append(line[1])
            release_speed.append(line[2])
            release_pos_x = line[3]
            release_pos_y = line[68]
            release_pos_z = line[4]
            release_pos.append(vector(float(release_pos_x),float(release_pos_y),float(release_pos_z)))
            player_name.append(line[5])
            event.append(line[8])
            result.append(line[9])
            balls.append(line[24])
            strikes.append(line[25])
            vx0 = line[44]
            vy0 = line[45]
            vz0 = line[46]
            release_vel.append(vector(float(vx0),float(vy0),float(vz0)))
            pitch_name.append(line[78])
            spin_axis.append(line[89])
            release_spin.append(line[56])
            
    return date, release_speed, release_pos, player_name, event, result, balls, strikes, release_vel, pitch_name, spin_axis, release_spin

date = [] 
release_speed = [] #in mph
release_pos = [] #vectors of x0, y0, v0
player_name = []
event = []
result = [] #result of the pitch
balls = [] 
strikes = [] 
pitch_name = [] #pitch type
release_vel = [] #vectors of vx0, vy0, and vz0
spin_axis = [] #spin axis in 2D x-z plane, such that 180 is pure backspin and 0 is pure topspin
release_spin = [] #spin rate of pitch (rpm)

date, release_speed, release_pos, player_name, event, result, balls, strikes, release_vel, pitch_name, spin_axis, release_spin = readData()

date_list = []
for x in date:
    if x not in date_list:
        date_list.append(x)

print("Pitcher:", player_name[0], "\nDates: ", date_list)
y = len(date)-1

for x in range(0,len(date)):
    result[y-x] = result[y-x].replace("_", " ")
    event[y-x] = event[y-x].replace("_", " ")
    if (all(a.isalpha() or a.isspace() for a in event[y-x]) and event[y-x]):
        event[y-x] = '(' + event[y-x] + ')'
        
def getCd(v):
    # calculate value of drag coefficient for a particular speed and case
    case = 2
    if case == 0:
        Cd = 0
    elif case == 1:
        Cd = 0.5
    elif case == 2:
        a = 0.36
        b = 0.14
        c = 0.27
        vc = 34
        chi = (v - vc)/4
        if chi < 0:
            Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2)
        else:
            Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2/4)
    else:
        Cd = 0.5
    
    return Cd

def arraymag(v):
    #calculate magnitude of an array
    return np.sqrt(np.dot(v,v))

def arrayhat(v):
    #calculate unit vector of an array
    return v/mag(v)

def arraycross(v1,v2):
    #calculate the cross product of two vectors
    return np.cross(v1,v2)

def model_magnus(d, tn):
    #return array of derivatives
        
    #data
    x = d[0]
    y = d[1]
    z = d[2]  
    vx = d[3]
    vy = d[4]
    vz = d[5]
    
    #derivatives
    rate = np.zeros(6) #derivatives
    rate[0] = vx
    rate[1] = vy
    rate[2] = vz
    
    #speed
    v = np.array([vx,vy,vz])
    vmag = arraymag(v)
    
    #calculate force and dv/dt
    Cd=getCd(vmag)
    b2 = 1/2*Cd*rho*A
    
    ωmag = float(srate) * 2*np.pi / 60 #rad/s
    ω = np.array([sin(ωmag), 0, cos(ωmag)])
    S = r*ωmag/vmag
    CL = 0.62*S**0.7
    alpha = 1/2*CL*rho*A*r/S
    
    FM = alpha*arraycross(ω,v) #magnus force
    
    rate[3] = (-b2*vmag*vx)/m + FM[0]/m
    rate[4] = (-b2*vmag*vy)/m + FM[1]/m
    rate[5] = (-b2*vmag*vz - (m*g))/m  + FM[2]/m
    
    return rate


def runSim(posvec,vel,spin_rate,spin_axis):
    global b2, alpha #need to change the value of b2 and alpha
    #time
    t = 0
    h= 0.0005
    Nsteps = int(10/h)
    
    results = []
    
    #convert f/s to m/s
    vel.x = vel.x*0.3048
    #print(vel.x)
    vel.y = vel.y*0.3048
    #print(vel.y)
    vel.z = vel.z*0.3048
    #print(vel.z)
    #convert deg to rad
    #print(spin_axis)
    theta = float(spin_axis)*np.pi/180 
    #print(theta)
    
    #print(spin_rate)
    global srate
    srate = spin_rate
    #print(srate)
    ωmag = float(spin_rate) * 2*np.pi / 60 #rad/s
    ω = np.array([sin(ωmag),0, cos(ωmag)])
    #print(ωmag)
    
   


    data = np.array([posvec.x,posvec.y,posvec.z,vel.x,vel.y,vel.z])
    
    #store trajectory for plotting or animation

    for n in range(0,Nsteps-1):

        #update data
        data = ode.RK4(model_magnus, data, t, h)
        
        #update t
        t=t+h

        #store data for plotting
        results.append(vec(data[0],data[1],data[2]))
    
    return results
    
    
scene1 = canvas(title="Pitch Tracking", autoscale=False)
scene1.background = color.gray(0.6)
ground = box(pos=vec(-1,-1,-10.3), size=vec(200,200,20), color=color.green)
home = box(pos=vec(0,0,-0.2), size=vec(0.43177968,0.43177968,0.0381), color=color.white)
pitchbox = box(pos=vec(0.3048,18.4404,0.24384), size=vec(0.6096,0.1524,0.0762))
scene1.center = vec(0,9.2202,0.1524)
scene1.camera.axis = vec(0.0190896, 18.0867, -1.64663)
scene1.camera.pos = vec(0.688913, -5.3547, 2.64525)
#scene1.camera.rotate(angle=1/2)
scene1.pause()

pitches = []

Nsteps = int(10/0.01)
posarray=[vec(0,0,0)]*len(date)
#print(posarray)

#parameters of the system

g = 9.8 #N/kg
rho = 1.2 #kg/m^3
mu = 1.8e-5 #kg/m/s
r = 74e-3/2 #74 mm diameter, 9.25" in circumference
A = np.pi*r**2 #cross-sectional area
m = 0.145 #kg
Cd = 0.35 #actually depends on speed
b2 = 1/2*Cd*rho*A #will change as Cd changes
S = 0.01 #will change as omega and v change
CL = 0 #will change with S
alpha = 1/2*CL*rho*A*r/S
y=len(date)-1

#print(len(date))
for x in range(0,11):
    #print(x)
    #print(posarray[0][0])
    posarray[y-x] = vec(release_pos[y-x].x*0.3048,release_pos[y-x].y*0.3048,release_pos[y-x].z*0.3048)
    positions = []
    #get simulation data
    data = runSim(posarray[y-x],release_vel[y-x],release_spin[y-x],spin_axis[y-x])
    for b in range(1,len(data)):
        positions.append(data[b-1])
    
    #show in visualization
    if(pitch_name[y-x]=="4-Seam Fastball" or pitch_name[x]=="2-Seam Fastball"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.red, make_trail=True)
    elif(pitch_name[y-x]=="Slider" or pitch_name[x]=="Sinker"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.yellow, make_trail=True)
    elif(pitch_name[y-x]=="Changeup"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.blue, make_trail=True)
    elif(pitch_name[y-x]=="Curveball"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.magenta, make_trail=True)
    else:
        ball = sphere(pos = posarray[y-x], radius = r, color = color.white, make_trail=True)
    pitches.append(ball)
    
    z=0
    while(pitches[x].pos.y >= 0.43177968 and pitches[x].pos.z >= 0 and z<len(positions)):
        rate(300)
        z=z+1
        pitches[x].pos = positions[z]
    
    #print(date[y-x])
    print("Pitch:", '{0: <18}'.format(pitch_name[y-x]), '{0: <3}'.format(release_speed[y-x]), "MPH   Count:", balls[y-x], "-", strikes[y-x], " Result:", result[y-x], event[y-x])
    #scene1.pause()
    pitches[x].clear_trail()
    pitches[x].visible = False
        
for c in range(len(pitches)):
    #print(pitches[c].pos)
    pitches[c].visible = True
    

Pitcher: Kershaw, Clayton 
Dates:  ['2021-04-28', '2021-04-23', '2021-04-17', '2021-04-11', '2021-04-06', '2021-04-01']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Pitch: 4-Seam Fastball    90.6 MPH   Count: 0 - 0  Result: ball 
Pitch: 4-Seam Fastball    90.7 MPH   Count: 1 - 0  Result: ball 
Pitch: 4-Seam Fastball    91.2 MPH   Count: 2 - 0  Result: hit into play (field out)
Pitch: 4-Seam Fastball    92.0 MPH   Count: 0 - 0  Result: hit into play (field out)
Pitch: 4-Seam Fastball    90.7 MPH   Count: 0 - 0  Result: ball 
Pitch: Slider             88.0 MPH   Count: 1 - 0  Result: foul 
Pitch: Curveball          76.0 MPH   Count: 1 - 1  Result: called strike 
Pitch: 4-Seam Fastball    91.1 MPH   Count: 1 - 2  Result: hit into play (single)
Pitch: Slider             89.8 MPH   Count: 0 - 0  Result: foul 
Pitch: Curveball          76.2 MPH   Count: 0 - 1  Result: ball 
Pitch: 4-Seam Fastball    91.3 MPH   Count: 1 - 1  Result: swinging strike 


## Data Set 2: Kenta Maeda, 04/01/21
This first data set uses pitch data from Minnesota Pitcher Kenta Maeda. This game was against the Milwaukee Brewers.

This set shows his first eight pitches of the game. Instead of removing each pitch after it finishes, it remains on the visualization, allowing us to see the spread of these pitches compared to each other, and the path each one traveled.

It's also interesting to note that the starting position for each pitch is slightly different. This is as expected - every pitcher is going to release pitches in more or less the same area, but the exact timing and position is almost impossible to duplicate between pitches.

Another note of interest is that the pitches in this data set start on the left side of the plate (as opposed to the right of the previous data set). This again shows us that the starting positions are accurate, as Maeda throws right-handed while Kershaw (in the previous set) is left-handed.

In [3]:
def readData(filename = "maeda_data.csv"):
    #default dataset is savant.csv, user can pass in whatever they want
    
    #creates the necessary lists
    date = [] 
    release_speed = [] #in mph
    release_pos = [] #vectors of x0, y0, v0
    player_name = []
    event = []
    result = [] #result of the pitch
    balls = [] #current # of balls
    strikes = [] #current # of strikes
    pitch_name = [] #pitch type
    release_vel = [] #vectors of vx0, vy0, and vz0
    spin_axis = [] #spin axis in 2D x-z plane, such that 180 is pure backspin and 0 is pure topspin
    release_spin = [] #spin rate of pitch (rpm)
    
    with open(filename) as file:
        rawdata = csv.reader(file, delimiter = ',')
        next(file)
        for line in rawdata:
            date.append(line[1])
            release_speed.append(line[2])
            release_pos_x = line[3]
            release_pos_y = line[68]
            release_pos_z = line[4]
            release_pos.append(vector(float(release_pos_x),float(release_pos_y),float(release_pos_z)))
            player_name.append(line[5])
            event.append(line[8])
            result.append(line[9])
            balls.append(line[24])
            strikes.append(line[25])
            vx0 = line[44]
            vy0 = line[45]
            vz0 = line[46]
            release_vel.append(vector(float(vx0),float(vy0),float(vz0)))
            pitch_name.append(line[78])
            spin_axis.append(line[89])
            release_spin.append(line[56])
            
    return date, release_speed, release_pos, player_name, event, result, balls, strikes, release_vel, pitch_name, spin_axis, release_spin

date = [] 
release_speed = [] #in mph
release_pos = [] #vectors of x0, y0, v0
player_name = []
event = []
result = [] #result of the pitch
balls = [] 
strikes = [] 
pitch_name = [] #pitch type
release_vel = [] #vectors of vx0, vy0, and vz0
spin_axis = [] #spin axis in 2D x-z plane, such that 180 is pure backspin and 0 is pure topspin
release_spin = [] #spin rate of pitch (rpm)

date, release_speed, release_pos, player_name, event, result, balls, strikes, release_vel, pitch_name, spin_axis, release_spin = readData()

date_list = []
for x in date:
    if x not in date_list:
        date_list.append(x)

print("Pitcher:", player_name[0], "\nDates: ", date_list)
y = len(date)-1

for x in range(0,len(date)):
    result[y-x] = result[y-x].replace("_", " ")
    event[y-x] = event[y-x].replace("_", " ")
    if (all(a.isalpha() or a.isspace() for a in event[y-x]) and event[y-x]):
        event[y-x] = '(' + event[y-x] + ')'
        
def getCd(v):
    # calculate value of drag coefficient for a particular speed and case
    case = 2
    if case == 0:
        Cd = 0
    elif case == 1:
        Cd = 0.5
    elif case == 2:
        a = 0.36
        b = 0.14
        c = 0.27
        vc = 34
        chi = (v - vc)/4
        if chi < 0:
            Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2)
        else:
            Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2/4)
    else:
        Cd = 0.5
    
    return Cd

def arraymag(v):
    #calculate magnitude of an array
    return np.sqrt(np.dot(v,v))

def arrayhat(v):
    #calculate unit vector of an array
    return v/mag(v)

def arraycross(v1,v2):
    #calculate the cross product of two vectors
    return np.cross(v1,v2)

def model_magnus(d, tn):
    #return array of derivatives
        
    #data
    x = d[0]
    y = d[1]
    z = d[2]  
    vx = d[3]
    vy = d[4]
    vz = d[5]
    
    #derivatives
    rate = np.zeros(6) #derivatives
    rate[0] = vx
    rate[1] = vy
    rate[2] = vz
    
    #speed
    v = np.array([vx,vy,vz])
    vmag = arraymag(v)
    
    #calculate force and dv/dt
    Cd=getCd(vmag)
    b2 = 1/2*Cd*rho*A
    
    ωmag = float(srate) * 2*np.pi / 60 #rad/s
    ω = np.array([sin(ωmag), 0, cos(ωmag)])
    S = r*ωmag/vmag
    CL = 0.62*S**0.7
    alpha = 1/2*CL*rho*A*r/S
    
    FM = alpha*arraycross(ω,v) #magnus force
    
    rate[3] = (-b2*vmag*vx)/m + FM[0]/m
    rate[4] = (-b2*vmag*vy)/m + FM[1]/m
    rate[5] = (-b2*vmag*vz - (m*g))/m  + FM[2]/m
    
    return rate


def runSim(posvec,vel,spin_rate,spin_axis):
    global b2, alpha #need to change the value of b2 and alpha
    #time
    t = 0
    h= 0.0005
    Nsteps = int(10/h)
    
    results = []
    
    #convert f/s to m/s
    vel.x = vel.x*0.3048
    vel.y = vel.y*0.3048
    vel.z = vel.z*0.3048
    #convert deg to rad
    theta = float(spin_axis)*np.pi/180 
    
    global srate
    srate = spin_rate
    ωmag = float(spin_rate) * 2*np.pi / 60 #rad/s
    ω = np.array([sin(ωmag),0, cos(ωmag)])

    data = np.array([posvec.x,posvec.y,posvec.z,vel.x,vel.y,vel.z])
    
    #store trajectory for plotting or animation

    for n in range(0,Nsteps-1):

        #update data
        data = ode.RK4(model_magnus, data, t, h)
        
        #update t
        t=t+h

        #store data for plotting
        results.append(vec(data[0],data[1],data[2]))
    
    return results
    
    
scene1 = canvas(title="Pitch Tracking", autoscale=False)
scene1.background = color.gray(0.6)
ground = box(pos=vec(-1,-1,-10.3), size=vec(200,200,20), color=color.green)
home = box(pos=vec(0,0,-0.2), size=vec(0.43177968,0.43177968,0.0381), color=color.white)
pitchbox = box(pos=vec(0.3048,18.4404,0.24384), size=vec(0.6096,0.1524,0.0762))
scene1.center = vec(0,9.2202,0.1524)
scene1.camera.axis = vec(0.0190896, 18.0867, -1.64663)
scene1.camera.pos = vec(0.688913, -5.3547, 2.64525)
#scene1.camera.rotate(angle=1/2)
scene1.pause()

pitches = []

Nsteps = int(10/0.01)
posarray=[vec(0,0,0)]*len(date)
#print(posarray)

#parameters of the system

g = 9.8 #N/kg
rho = 1.2 #kg/m^3
mu = 1.8e-5 #kg/m/s
r = 74e-3/2 #74 mm diameter, 9.25" in circumference
A = np.pi*r**2 #cross-sectional area
m = 0.145 #kg
Cd = 0.35 #actually depends on speed
b2 = 1/2*Cd*rho*A #will change as Cd changes
S = 0.01 #will change as omega and v change
CL = 0 #will change with S
alpha = 1/2*CL*rho*A*r/S
y=len(date)-1

allpos = []
for x in range(0,8):
    posarray[y-x] = vec(release_pos[y-x].x*0.3048,release_pos[y-x].y*0.3048,release_pos[y-x].z*0.3048)
    positions = []
    #get simulation data
    data = runSim(posarray[y-x],release_vel[y-x],release_spin[y-x],spin_axis[y-x])
    for b in range(1,len(data)):
        positions.append(data[b-1])
    
    #show in visualization
    if(pitch_name[y-x]=="4-Seam Fastball" or pitch_name[x]=="2-Seam Fastball"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.red, make_trail=True)
    elif(pitch_name[y-x]=="Slider" or pitch_name[y-x]=="Sinker"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.yellow, make_trail=True)
    elif(pitch_name[y-x]=="Changeup"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.blue, make_trail=True)
    elif(pitch_name[y-x]=="Curveball"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.magenta, make_trail=True)
    else:
        ball = sphere(pos = posarray[y-x], radius = r, color = color.white, make_trail=True)
    pitches.append(ball)
    
    z=0
    pitchsets = []
    for f in range(len(positions)):
        pitchsets.append(positions[f])
    allpos.append(pitchsets)
    
    print("Pitch:", '{0: <18}'.format(pitch_name[y-x]), '{0: <3}'.format(release_speed[y-x]), "MPH   Count:", balls[y-x], "-", strikes[y-x], " Result:", result[y-x], event[y-x])
        
    while(pitches[x].pos.y >= 0.43177968 and pitches[x].pos.z >= 0 and z<len(positions)):
        rate(400)
        z=z+1
        pitches[x].pos = positions[z]
    

Pitcher: Maeda, Kenta 
Dates:  ['2021-04-14', '2021-04-07', '2021-04-01']


<IPython.core.display.Javascript object>

Pitch: Sinker             91.2 MPH   Count: 0 - 0  Result: called strike 
Pitch: 4-Seam Fastball    92.2 MPH   Count: 0 - 1  Result: foul 
Pitch: Changeup           85.8 MPH   Count: 0 - 2  Result: ball 
Pitch: Changeup           85.7 MPH   Count: 1 - 2  Result: foul tip (strikeout)
Pitch: Sinker             92.7 MPH   Count: 0 - 0  Result: ball 
Pitch: Sinker             92.4 MPH   Count: 1 - 0  Result: ball 
Pitch: Slider             83.9 MPH   Count: 2 - 0  Result: swinging strike 
Pitch: Slider             85.7 MPH   Count: 2 - 1  Result: swinging strike 


## Data Set 3: Aroldis Chapman, 04/05/21
This first data set uses pitch data from New York Yankees Closer Aroldis Chapman. This game was against the Baltimore Orioles.

It shows all 19 of Chapman's pitches in the outing, overlayed against eachother, as in the previous data set.

In [4]:
def readData(filename = "chapman_data.csv"):
    #default dataset is savant.csv, user can pass in whatever they want
    
    #creates the necessary lists
    date = [] 
    release_speed = [] #in mph
    release_pos = [] #vectors of x0, y0, v0
    player_name = []
    event = []
    result = [] #result of the pitch
    balls = [] #current # of balls
    strikes = [] #current # of strikes
    pitch_name = [] #pitch type
    release_vel = [] #vectors of vx0, vy0, and vz0
    spin_axis = [] #spin axis in 2D x-z plane, such that 180 is pure backspin and 0 is pure topspin
    release_spin = [] #spin rate of pitch (rpm)
    
    with open(filename) as file:
        rawdata = csv.reader(file, delimiter = ',')
        next(file)
        for line in rawdata:
            date.append(line[1])
            release_speed.append(line[2])
            release_pos_x = line[3]
            release_pos_y = line[68]
            release_pos_z = line[4]
            release_pos.append(vector(float(release_pos_x),float(release_pos_y),float(release_pos_z)))
            player_name.append(line[5])
            event.append(line[8])
            result.append(line[9])
            balls.append(line[24])
            strikes.append(line[25])
            vx0 = line[44]
            vy0 = line[45]
            vz0 = line[46]
            release_vel.append(vector(float(vx0),float(vy0),float(vz0)))
            pitch_name.append(line[78])
            spin_axis.append(line[89])
            release_spin.append(line[56])
            
    return date, release_speed, release_pos, player_name, event, result, balls, strikes, release_vel, pitch_name, spin_axis, release_spin

date = [] 
release_speed = [] #in mph
release_pos = [] #vectors of x0, y0, v0
player_name = []
event = []
result = [] #result of the pitch
balls = [] 
strikes = [] 
pitch_name = [] #pitch type
release_vel = [] #vectors of vx0, vy0, and vz0
spin_axis = [] #spin axis in 2D x-z plane, such that 180 is pure backspin and 0 is pure topspin
release_spin = [] #spin rate of pitch (rpm)

date, release_speed, release_pos, player_name, event, result, balls, strikes, release_vel, pitch_name, spin_axis, release_spin = readData()

date_list = []
for x in date:
    if x not in date_list:
        date_list.append(x)

print("Pitcher:", player_name[0], "\nDates: ", date_list)
y = len(date)-1

for x in range(0,len(date)):
    result[y-x] = result[y-x].replace("_", " ")
    event[y-x] = event[y-x].replace("_", " ")
    if (all(a.isalpha() or a.isspace() for a in event[y-x]) and event[y-x]):
        event[y-x] = '(' + event[y-x] + ')'
        
def getCd(v):
    # calculate value of drag coefficient for a particular speed and case
    case = 2
    if case == 0:
        Cd = 0
    elif case == 1:
        Cd = 0.5
    elif case == 2:
        a = 0.36
        b = 0.14
        c = 0.27
        vc = 34
        chi = (v - vc)/4
        if chi < 0:
            Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2)
        else:
            Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2/4)
    else:
        Cd = 0.5
    
    return Cd

def arraymag(v):
    #calculate magnitude of an array
    return np.sqrt(np.dot(v,v))

def arrayhat(v):
    #calculate unit vector of an array
    return v/mag(v)

def arraycross(v1,v2):
    #calculate the cross product of two vectors
    return np.cross(v1,v2)

def model_magnus(d, tn):
    #return array of derivatives
        
    #data
    x = d[0]
    y = d[1]
    z = d[2]  
    vx = d[3]
    vy = d[4]
    vz = d[5]
    
    #derivatives
    rate = np.zeros(6) #derivatives
    rate[0] = vx
    rate[1] = vy
    rate[2] = vz
    
    #speed
    v = np.array([vx,vy,vz])
    vmag = arraymag(v)
    
    #calculate force and dv/dt
    Cd=getCd(vmag)
    b2 = 1/2*Cd*rho*A
    
    ωmag = float(srate) * 2*np.pi / 60 #rad/s
    ω = np.array([sin(ωmag), 0, cos(ωmag)])
    S = r*ωmag/vmag
    CL = 0.62*S**0.7
    alpha = 1/2*CL*rho*A*r/S
    
    FM = alpha*arraycross(ω,v) #magnus force
    
    rate[3] = (-b2*vmag*vx)/m + FM[0]/m
    rate[4] = (-b2*vmag*vy)/m + FM[1]/m
    rate[5] = (-b2*vmag*vz - (m*g))/m  + FM[2]/m
    
    return rate


def runSim(posvec,vel,spin_rate,spin_axis):
    global b2, alpha #need to change the value of b2 and alpha
    #time
    t = 0
    h= 0.0005
    Nsteps = int(10/h)
    
    results = []
    
    #convert f/s to m/s
    vel.x = vel.x*0.3048
    vel.y = vel.y*0.3048
    vel.z = vel.z*0.3048
    #convert deg to rad
    theta = float(spin_axis)*np.pi/180 
    
    global srate
    srate = spin_rate
    ωmag = float(spin_rate) * 2*np.pi / 60 #rad/s
    ω = np.array([sin(ωmag),0, cos(ωmag)])

    data = np.array([posvec.x,posvec.y,posvec.z,vel.x,vel.y,vel.z])
    
    #store trajectory for plotting or animation

    for n in range(0,Nsteps-1):

        #update data
        data = ode.RK4(model_magnus, data, t, h)
        
        #update t
        t=t+h

        #store data for plotting
        results.append(vec(data[0],data[1],data[2]))
    
    return results
    
    
scene1 = canvas(title="Pitch Tracking", autoscale=False)
scene1.background = color.gray(0.6)
ground = box(pos=vec(-1,-1,-10.3), size=vec(200,200,20), color=color.green)
home = box(pos=vec(0,0,-0.2), size=vec(0.43177968,0.43177968,0.0381), color=color.white)
pitchbox = box(pos=vec(0.3048,18.4404,0.24384), size=vec(0.6096,0.1524,0.0762))
scene1.center = vec(0,9.2202,0.1524)
scene1.camera.axis = vec(0.0190896, 18.0867, -1.64663)
scene1.camera.pos = vec(0.688913, -5.3547, 2.64525)
#scene1.camera.rotate(angle=1/2)
scene1.pause()

pitches = []

Nsteps = int(10/0.01)
posarray=[vec(0,0,0)]*len(date)
#print(posarray)

#parameters of the system

g = 9.8 #N/kg
rho = 1.2 #kg/m^3
mu = 1.8e-5 #kg/m/s
r = 74e-3/2 #74 mm diameter, 9.25" in circumference
A = np.pi*r**2 #cross-sectional area
m = 0.145 #kg
Cd = 0.35 #actually depends on speed
b2 = 1/2*Cd*rho*A #will change as Cd changes
S = 0.01 #will change as omega and v change
CL = 0 #will change with S
alpha = 1/2*CL*rho*A*r/S
y=len(date)-1

allpos = []
#this can be adjusted to alter the amount of pitches shown
#it can be set to range(len(dates)), for example, to show the entire data set
for x in range(0,19):
    posarray[y-x] = vec(release_pos[y-x].x*0.3048,release_pos[y-x].y*0.3048,release_pos[y-x].z*0.3048)
    positions = []
    #get simulation data
    data = runSim(posarray[y-x],release_vel[y-x],release_spin[y-x],spin_axis[y-x])
    for b in range(1,len(data)):
        positions.append(data[b-1])
    
    #show in visualization
    if(pitch_name[y-x]=="4-Seam Fastball" or pitch_name[x]=="2-Seam Fastball"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.red, make_trail=True)
    elif(pitch_name[y-x]=="Slider" or pitch_name[y-x]=="Sinker"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.yellow, make_trail=True)
    elif(pitch_name[y-x]=="Changeup"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.blue, make_trail=True)
    elif(pitch_name[y-x]=="Curveball"):
        ball = sphere(pos = posarray[y-x], radius = r, color = color.magenta, make_trail=True)
    else:
        ball = sphere(pos = posarray[y-x], radius = r, color = color.white, make_trail=True)
    pitches.append(ball)
    
    z=0
    pitchsets = []
    for f in range(len(positions)):
        pitchsets.append(positions[f])
    allpos.append(pitchsets)
    
    print("Pitch:", '{0: <18}'.format(pitch_name[y-x]), '{0: <3}'.format(release_speed[y-x]), "MPH   Count:", balls[y-x], "-", strikes[y-x], " Result:", result[y-x], event[y-x])
        
    while(pitches[x].pos.y >= 0.43177968 and pitches[x].pos.z >= 0 and z<len(positions)):
        rate(400)
        z=z+1
        pitches[x].pos = positions[z]
    

Pitcher: Chapman, Aroldis 
Dates:  ['2021-05-01', '2021-04-29', '2021-04-23', '2021-04-22', '2021-04-20', '2021-04-12', '2021-04-11', '2021-04-07', '2021-04-05']


<IPython.core.display.Javascript object>

Pitch: 4-Seam Fastball    98.0 MPH   Count: 0 - 0  Result: swinging strike 
Pitch: 4-Seam Fastball    98.1 MPH   Count: 0 - 1  Result: foul tip 
Pitch: 4-Seam Fastball    100.2 MPH   Count: 0 - 2  Result: ball 
Pitch: Slider             84.4 MPH   Count: 1 - 2  Result: foul 
Pitch: Slider             85.4 MPH   Count: 1 - 2  Result: swinging strike (strikeout)
Pitch: Slider             84.6 MPH   Count: 0 - 0  Result: called strike 
Pitch: Slider             85.4 MPH   Count: 0 - 1  Result: swinging strike 
Pitch: Split-Finger       90.5 MPH   Count: 0 - 2  Result: swinging strike (strikeout)
Pitch: 4-Seam Fastball    98.8 MPH   Count: 0 - 0  Result: ball 
Pitch: 4-Seam Fastball    99.4 MPH   Count: 1 - 0  Result: ball 
Pitch: Slider             83.3 MPH   Count: 2 - 0  Result: called strike 
Pitch: Slider             85.8 MPH   Count: 2 - 1  Result: called strike 
Pitch: 4-Seam Fastball    101.3 MPH   Count: 2 - 2  Result: ball 
Pitch: Slider             86.0 MPH   Count: 3 - 2  Resul

## Validation
I was able to validate most of the results of this project using baseball savant's [3D pitch visualization](https://baseballsavant.mlb.com/visuals/pitch3d?player_id=477132#v=1&mainView=tracking&pov=umpire&hitterSide=all&marks=none&plays1=all&plays2=all). This showed me what most of the pitches actually looked like, so it was a solid metric to compare my results against.

## Shortcomings
This model is not perfect. One of the biggest reasons why is that it assumes that the baseball is perfectly spherical, which is not actually the case. Baseballs have seams running along them that often have a large effect on the motion of a baseball. 

For example, a knuckleball is a pitch that this program wil have trouble handling. When a pitcher throws a knuckleball, they attempt to put as little spin on the baseball as possible. As a result, the pitch moves extremely erratically, as the drag caused by the seams are the driving force behind the pitch's motion.

Furthermore, some pitches, especially those that break hard and late (that curve sharply when they are near to home plate) do not always have as much motion as they should. This is likely due to the time step assumed by the program; rather than having the baseball follow a smooth trajectory, it actually moves it in small jumps that are small enough as to seem fluid to the human eye when run through the simulation. As a result, it is not a perfect representation of the baseball's motion. 

Decreasing the time step would be possible to give a more accurate trajectory, however, doing so makes the program run significantly slower. The current time step strikes a good balance between runtime and accuracy.
